In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from parallel_pandas import ParallelPandas
import os

In [5]:
from funcs import *

In [6]:
sp = mdf_np(100000) ## initialize dataset. 

In [8]:
sp, i = run_gale_shapley(sp) ## run gale-shapley algorithm

In [11]:
sp

,0,1,2,student_id,applications,N,k,matched,rank1,rank2,rank3,underdemanded
0,45066,45066,45066,0,2,20,1,False,0.823729,0.823729,0.823729,False
1,99974,54369,8997,1,0,19,1,False,0.900695,0.949901,0.355195,False
2,12733,55921,25163,2,0,18,1,False,0.596219,0.833194,0.500717,False
3,41418,49903,63688,3,0,16,1,False,0.496904,0.563762,0.853821,True
4,38693,9808,26564,4,0,18,1,False,0.506695,0.432942,0.834421,False
...,...,...,...,...,...,...,...,...,...,...,...,...
83979,48490,67429,78429,99994,0,16,1,False,0.870285,0.647651,0.937457,True
83980,67483,37177,91420,99995,0,16,1,False,0.155439,0.659249,0.831656,True
83981,33628,84025,16406,99996,0,16,1,False,0.737563,0.706841,0.720876,True
83982,65262,65213,56706,99997,0,17,1,False,0.779385,0.633151,0.994172,False


Between Rounds of EADAM.

We now have a dataframe that shows all matches after 1 round. We now want to delete the student/school pairs where the school was undermatched to. So all students who never faced k>2 are deleted as well as the schools they matched to. We also remove all schools that received no matches, all students that went unmatched. This should leave us with a dataframe that contains all remaining students, and their school choices. Now, you'll note that we've only removed students. So a school that we've deleted, though it may appear in the dataframe, will not be reached. The reason for this is that we should not get to that school. 



In [13]:
def remove_ud(sp_f, sp):
    '''
    A function that takes as input a final set of matchings and determines which schools have been underdemanded!
    The underdemanded schools are all the schools that have either (1) gone unmatched (as they were never demanded)
    or (2) have been demanded by only a single student. 
    '''
    ## All underdemanded schools underdemanded in the sense of never matched have
    # already deleted self. Now delete others.
    sp_f = sp_f[sp_f['underdemanded'] == False] ## delete all agents that wound up at underdemanded schools
    ## Now we want to back to the original school list and grab only those agents that wound up at demanded schools.
    sp_new = sp[sp['student_id'].isin(sp_f['student_id'])] ## Holds on only to those students that ended up matched to a demanded school!
    return sp_new


In [14]:
sp = mdf_np(100) ## initialize dataset.

In [18]:
sp_f, i = run_gale_shapley(sp) ## run gale-shapley algorithm

In [19]:
sp_f

,0,1,2,student_id,applications,N,k,matched,rank1,rank2,rank3,underdemanded
0,93,79,10,1,0,6,1,False,0.818331,0.627725,0.754176,False
1,95,70,70,2,1,4,1,False,0.448338,0.130274,0.130274,False
2,33,62,62,3,1,4,1,False,0.513956,0.739149,0.739149,False
3,63,57,71,4,0,5,1,False,0.954176,0.909617,0.295412,True
4,86,59,59,5,1,4,1,False,0.959061,0.946543,0.946543,False
...,...,...,...,...,...,...,...,...,...,...,...,...
77,28,33,59,94,0,6,1,False,0.972825,0.419416,0.726140,False
78,85,85,85,96,2,6,1,False,0.565828,0.565828,0.565828,False
79,59,82,77,97,0,5,1,False,0.841635,0.118523,0.187648,True
80,78,20,20,98,1,4,1,False,0.934468,0.382233,0.382233,False


In [23]:
sp2 = remove_ud(sp_f, sp).reset_index(drop=True)

In [24]:
sp_f2 = run_gale_shapley(sp2)[0]

In [25]:
sp_f2

,0,1,2,student_id,applications,N,k,matched,rank1,rank2,rank3,underdemanded
0,93,79,10,1,0,6,1,False,0.818331,0.627725,0.754176,False
1,95,70,70,2,1,4,1,False,0.448338,0.130274,0.130274,False
2,33,62,62,3,1,4,1,False,0.513956,0.739149,0.739149,False
3,86,59,59,5,1,4,1,False,0.959061,0.946543,0.946543,False
4,45,71,93,6,0,6,1,False,0.295798,0.262032,0.317786,False
...,...,...,...,...,...,...,...,...,...,...,...,...
65,8,34,90,93,0,7,1,False,0.979065,0.794683,0.107223,False
66,28,33,59,94,0,5,1,False,0.972825,0.419416,0.726140,True
67,85,85,85,96,2,6,1,False,0.565828,0.565828,0.565828,False
68,78,20,20,98,1,4,1,False,0.934468,0.382233,0.382233,False


In [75]:
def comparison(sp_f, sp):
    X = pd.testing.assert_frame_equal(sp_f[[0, 'student_id']].reset_index(drop = True), sp[[0, 'student_id']].reset_index(drop=True), check_dtype=False)
    if X  is None:
        X = False
    else:
        X = True
    return X

In [76]:
def compare(sp_f, sp):
    '''
    This function takes two sets of matchings and compares them. This is used
    to determine whether we should "end" the EADAM algorithm or not.
    ______
    sp_f: round n+1 dataframe. 
    sp: prior round matchings.  
    '''
    X = True ## initialize boolean to say that they are the same
    if len(sp_f) != len(sp): ## if the number of matchings is different, then they are not the same
        X = True
    else:
        X = comparison(sp_f, sp) ## if the number of matchings is the same, then we need to compare them
    return X

In [78]:
compare(sp_f, sp)

True

In [86]:
pd.options.mode.chained_assignment = None  # default='warn'

In [87]:
def EADAM(sp):
    '''
    This function runs the EADAM algorithm 
    '''
    change = True ## initialize boolean to say that there has been a change
    j = 0 ## initialize counter, counts number of rounds in EADAM (number of times we run GS)
    iter_list = [] ## initialize list to hold the number of iterations in each GS run.
    while change == True:
        sp_f, i = run_gale_shapley(sp)
        if j == 0:
            gs_result = sp_f
        sp_f = remove_ud(sp_f, sp)
        change = compare(sp_f, sp)
        sp = sp_f
        iter_list.append(i)
        j += 1
    return sp_f, gs_result, iter_list, j

In [92]:
sp = mdf_np(1000000)

In [93]:
sp_f, gs_result, iter_list, j = EADAM(sp)

In [94]:
gs_result

,0,1,2,student_id,applications,N,k,matched,rank1,rank2,rank3,underdemanded
0,909668,147053,274619,0,0,24,1,False,0.909152,0.487099,0.962447,False
1,207467,795067,348971,1,0,21,1,False,0.082543,0.714019,0.259097,True
2,981163,407363,407363,2,1,22,1,False,0.824157,0.882912,0.882912,False
3,193851,998416,998416,3,1,23,1,False,0.438670,0.924898,0.924898,False
4,81041,81041,81041,4,2,21,1,False,0.582467,0.582467,0.582467,False
...,...,...,...,...,...,...,...,...,...,...,...,...
840128,903866,34255,34255,999991,1,22,1,False,0.954726,0.996594,0.996594,False
840129,869266,635303,304510,999994,0,24,1,False,0.994735,0.684344,0.954038,False
840130,994004,579607,282903,999995,0,23,1,False,0.910544,0.897662,0.179601,False
840131,264575,40529,495051,999997,0,22,1,False,0.614582,0.955285,0.081452,False


In [95]:
sp_f

,0,1,2,student_id,applications,N,k,matched,rank1,rank2,rank3,underdemanded
0,909668,147053,274619,0,0,5.0,1,False,0.909152,0.487099,0.962447,False
2,981163,407363,407363,2,1,4.0,2,True,0.824157,0.882912,0.882912,False
3,193851,998416,998416,3,1,2.0,1,False,0.438670,0.924898,0.924898,False
4,81041,81041,81041,4,2,4.0,1,False,0.582467,0.582467,0.582467,False
5,125526,878971,878971,5,1,3.0,1,False,0.877035,0.781663,0.781663,False
...,...,...,...,...,...,...,...,...,...,...,...,...
999989,528697,643238,643238,999989,1,NaN,1,False,0.977919,0.086213,0.086213,False
999991,903866,34255,34255,999991,1,NaN,1,False,0.954726,0.996594,0.996594,False
999994,869266,635303,304510,999994,0,NaN,1,False,0.994735,0.684344,0.954038,False
999995,994004,579607,282903,999995,0,NaN,1,False,0.910544,0.897662,0.179601,False


Ultimately, we want to compare the one-shot gs results. We need to save the matchings that get removed because they were demanded only the one time. That way, at the end, we can reconstruct a frankenstein matching made up of the last dataframe of the EADAM algo concatenated with all the matchings that were removed. As we gradually removed them.  

In [ ]:
def save_matches(sp_f, undermatched_matches):
    '''
    At the end of every Gale-Shapley run, we are removing underdemanded schools.
    Note that these are still matches. And our end goal is to compare a one-shot GS
    to the EADAM algorithm. So we need to add back in the underdemanded schools.
    '''
    sp_f[sp_f['undermatched'] == True] = undermatched_matches
